In [135]:
import os
import numpy as np
import pandas as pd
import json
from matplotlib import pyplot

pd.set_option('display.max_columns', None)

In [150]:
# Open file
FILE_DIR = "../"
tmdb_path = os.path.join(FILE_DIR,"data","the-movies-dataset")
movies_metadata_dir = os.path.join(tmdb_path, "movies_metadata.csv")
movies_metadata = pd.read_csv(movies_metadata_dir)
movies_metadata

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,0.072051,/jldsYflnId4tTWPx8es3uzsB1I8.jpg,[],"[{'iso_3166_1': 'IR', 'name': 'Iran'}]",NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,0.178241,/xZkmxsNmYXJbKVsTRLLx3pqGHx7.jpg,"[{'name': 'Sine Olivia', 'id': 19653}]","[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg,"[{'name': 'American World Pictures', 'id': 6165}]","[{'iso_3166_1': 'US', 'name': 'United States o...",2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,Fa

In [151]:
# Delete movies where both budget and revenue are zero
# This helps GMMs as there are many movies that are clustered around (0,0)
movies_metadata = movies_metadata[movies_metadata['revenue'] > 0]
movies_metadata = movies_metadata[movies_metadata['budget'] != '0']
movies_metadata = movies_metadata.reset_index(drop=True)

In [152]:
# One hot encoding for features

# Does the movie belong to a series?
movies_metadata['belongs_to_collection'] = movies_metadata['belongs_to_collection'].isna().apply(lambda x: 0 if x==False else 1)

# Extract movie genres
movies_metadata['genres'] = movies_metadata['genres'].apply(lambda x: eval(x))
movies_metadata['genres'] = movies_metadata['genres'].apply(lambda x: [genre['name'] for genre in x])

# Categorize top 7 genres: Drama, Comedy, Thriller, Action, Adventure, Family,
movies_metadata['drama'] = movies_metadata['genres'].apply(lambda x: 1 if 'Drama' in x else 0)
movies_metadata['comedy'] = movies_metadata['genres'].apply(lambda x: 1 if 'Comedy' in x else 0)
movies_metadata['thriller'] = movies_metadata['genres'].apply(lambda x: 1 if 'Thriller' in x else 0)
movies_metadata['action'] = movies_metadata['genres'].apply(lambda x: 1 if 'Action' in x else 0)
movies_metadata['romance'] = movies_metadata['genres'].apply(lambda x: 1 if 'Romance' in x else 0)
movies_metadata['adventure'] = movies_metadata['genres'].apply(lambda x: 1 if 'Adventure' in x else 0)
movies_metadata['family'] = movies_metadata['genres'].apply(lambda x: 1 if 'Family' in x else 0)

# Categorize films into released in 2010s, 2000s, 1990s, 1980s_and_below
movies_metadata['release_year'] = movies_metadata['release_date'].apply(lambda x: int(x[:4]) if isinstance(x, str) else 0)
movies_metadata['release_2010s'] = movies_metadata['release_year'].apply(lambda x: 1 if x >= 2010 else 0)
movies_metadata['release_2000s'] = movies_metadata['release_year'].apply(lambda x: 1 if (x >= 2000 and x < 2010) else 0)
movies_metadata['release_1990s'] = movies_metadata['release_year'].apply(lambda x: 1 if (x >= 1990 and x < 2000) else 0)
movies_metadata['release_1980s_below'] = movies_metadata['release_year'].apply(lambda x: 1 if (x < 1990) else 0)

In [153]:
# Extract sentiment score of overviews and taglines using
# VADER (Valence Aware Dictionary and sEntiment Reasoner)
import time
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyser = SentimentIntensityAnalyzer()

def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
    return score['compound']

start_time = time.time()
movies_metadata['overview_vader'] = movies_metadata['overview'].apply(lambda x: sentiment_analyzer_scores(x) if isinstance(x, str) else 0)
print("Done with overviews in:", time.time() - start_time)
movies_metadata['tagline_vader'] = movies_metadata['tagline'].apply(lambda x: sentiment_analyzer_scores(x) if isinstance(x, str) else 0)
print("Done with tagline in:", time.time() - start_time)

Done with overviews in: 3.9075980186462402
Done with tagline in: 4.42809796333313


In [154]:
# Remove unwanted features
movies_metadata = movies_metadata.drop([
    'adult', 
    'genres', 
    'homepage', 
    'imdb_id', 
    'original_language', 
    'overview', 
    'poster_path', 
    'production_companies',
    'production_countries',
    'release_date',
    'spoken_languages',
    'status',
    'tagline',
    'title',
    'video',
    'release_year'
], axis=1)

In [155]:
# Check after data processing
movies_metadata

,belongs_to_collection,budget,id,original_title,popularity,revenue,runtime,vote_average,vote_count,drama,comedy,thriller,action,romance,adventure,family,release_2010s,release_2000s,release_1990s,release_1980s_below,overview_vader,tagline_vader
0,0,30000000,862,Toy Story,21.9469,373554033.0,81.0,7.7,5415.0,0,1,0,0,0,0,1,0,0,1,0,0.1280,0.0000
1,1,65000000,8844,Jumanji,17.0155,262797249.0,104.0,6.9,2413.0,0,0,0,0,0,1,1,0,0,1,0,-0.4588,0.5411
2,1,16000000,31357,Waiting to Exhale,3.85949,81452156.0,127.0,6.1,34.0,1,1,0,0,1,0,0,0,0,1,0,0.8860,0.5812
3,1,60000000,949,Heat,17.9249,187436818.0,170.0,7.7,1886.0,1,0,1,1,0,0,0,0,0,1,0,-0.8750,-0.5423
4,1,35000000,9091,Sudden Death,5.23158,64350171.0,106.0,5.5,174.0,0,0,1,1,0,1,0,0,0,1,0,0.5106,-0.5267
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5376,1,12000000,24049,சிவாஜி,1.323587,19000000.0,185.0,6.9,25.0,1,1,0,1,0,0,0,0,1,0,0,0.6712,0.0000
5377,1,750000,280422,Все и сразу,0.201582,3.0,0.0,6.0,4.0,0,1,0,0,0,0,0,1,0,0,0,0.0000,0.0000
5378,1,800000,62757,Dikari,0.903061,1328612.0,100.0,5.8,6.0,1,1,0,0,0,0,0,0,1,0,0,0.7037,0.0000
5379,1,2000000,63281,Про любоff,0.121844,1268793.0,107.0,4.0,3.0,1,0,0,0,1,0,0,1,0,0,0,0.0000,0.0000


In [157]:
# Save
movies_metadata.to_csv(os.path.join(tmdb_path, "movies_metadata_processed.csv"))